# Arixiv Search with OpenCLIP and LanceDB

In this example we'll build a Arxiv Search or a recommender based on semantic search using LanceDB. We'll also compare the results with keyword based saerch on Nomic's atlast


## OpenCLIP

![CLIP (1)](https://github.com/lancedb/vectordb-recipes/assets/15766192/11b3b900-0bcb-4a4a-8fd4-804611c85972)


OpenCLIP an open source implementation of OpenAI's CLIP (Contrastive Language-Image Pre-training) as is available with various backends

In [1]:
# SETUP
!pip install lancedb open_clip_torch arxiv --q

In [2]:
!pip install pandas

## Creating table from arxiv API

### Embedding Paper Summary using CLIP


In [3]:
import torch
import open_clip
import pandas as pd
from open_clip import tokenizer
from tqdm import tqdm
from collections import defaultdict
import arxiv
import lancedb


def embed_func_clip(text):
    model, _, preprocess = open_clip.create_model_and_transforms(
        "ViT-B-32", pretrained="laion2b_s34b_b79k"
    )
    tokenizer = open_clip.get_tokenizer("ViT-B-32")
    with torch.no_grad():
        text_features = model.encode_text(tokenizer(text))
    return text_features

### Create a DataFrame of the desired length

Here we'll use arxiv python utility to interact with arxiv api and get the document data

In [4]:
def get_arxiv_df(embed_func, length=100):
    results = arxiv.Search(
        query="cat:cs.AI OR cat:cs.CV OR cat:stat.ML",
        max_results=length,
        sort_by=arxiv.SortCriterion.Relevance,
        sort_order=arxiv.SortOrder.Descending,
    ).results()
    df = defaultdict(list)
    for result in tqdm(results, total=length):
        try:
            df["title"].append(result.title)
            df["summary"].append(result.summary)
            df["authors"].append(str(result.authors))
            df["url"].append(result.entry_id)
            df["vector"].append(embed_func(result.summary).tolist()[0])

        except Exception as e:
            print("error: ", e)

    return pd.DataFrame(df)

In [5]:
LENGTH = 100  # Reduce the size for demo


def create_table():
    db = lancedb.connect("db")
    df = get_arxiv_df(embed_func_clip, LENGTH)

    tbl = db.create_table("arxiv", data=df, mode="overwrite")

    return tbl

In [17]:
tbl = create_table()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [07:42<00:00,  4.62s/it]


In [6]:
import lancedb

db = lancedb.connect("db")

if "arxiv" not in db.table_names():
    tbl = create_table()
else:
    tbl = db.open_table("arxiv")

In [7]:
len(tbl)

100

## Semantic Search by concepts or summary

In [8]:
from IPython.display import display, HTML


def search_table(query, embed_func=embed_func_clip, lim=3):
    db = lancedb.connect("db")
    tbl = db.open_table("arxiv")

    embs = embed_func(query)

    return tbl.search(embs.tolist()[0]).limit(3).to_pandas()

In [9]:
len(tbl)

100

In [10]:
# MobileSAM paper abstract 2nd half
query = """
Many of such applications need to be run on resource-constraint edge devices,
like mobile phones. In this work, we aim to make SAM mobile-friendly by replacing the heavyweight
image encoder with a lightweight one. A naive way to train such a new SAM as in the original SAM
paper leads to unsatisfactory performance, especially when limited training sources are available. We
find that this is mainly caused by the coupled optimization of the image encoder and mask decoder,
motivated by which we propose decoupled distillation. Concretely, we distill the knowledge from
the heavy image encoder (ViT-H in the original SAM) to a lightweight image encoder, which can be
automatically compatible with the mask decoder in the original SAM. The training can be completed
on a single GPU within less than one day, and the resulting lightweight SAM is termed MobileSAM
which is more than 60 times smaller yet performs on par with the original SAM. For inference speed,
With a single GPU, MobileSAM runs around 10ms per image: 8ms on the image encoder and 4ms
on the mask decoder. With superior performance, our MobileSAM is around 5 times faster than the
concurrent FastSAM and 7 times smaller, making it more suitable for mobile applications. Moreover,
we show that MobileSAM can run relatively smoothly on CPU
"""

result = search_table(query)

result.pop("vector")
display(HTML(result.to_html()))

open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

C:\Users\FISCLOUDS\AppData\Local\Temp\ipykernel_26876\404315364.py:9: DeprecatedWarning: to_df is deprecated as of 0.3.1 and will be removed in 0.4.0. Use the bar function instead
  return tbl.search(embs.tolist()[0]).limit(3).to_df()


,title,summary,authors,url,_distance
0,Twin-GAN -- Unpaired Cross-Domain Image Translation with Weight-Sharing GANs,"We present a framework for translating unlabeled images from one domain into\nanalog images in another domain. We employ a progressively growing\nskip-connected encoder-generator structure and train it with a GAN loss for\nrealistic output, a cycle consistency loss for maintaining same-domain\ntranslation identity, and a semantic consistency loss that encourages the\nnetwork to keep the input semantic features in the output. We apply our\nframework on the task of translating face images, and show that it is capable\nof learning semantic mappings for face images with no supervised one-to-one\nimage mapping.",[arxiv.Result.Author('Jerry Li')],http://arxiv.org/abs/1809.00946v1,37.476677
1,TADAM: Task dependent adaptive metric for improved few-shot learning,"Few-shot learning has become essential for producing models that generalize\nfrom few examples. In this work, we identify that metric scaling and metric\ntask conditioning are important to improve the performance of few-shot\nalgorithms. Our analysis reveals that simple metric scaling completely changes\nthe nature of few-shot algorithm parameter updates. Metric scaling provides\nimprovements up to 14% in accuracy for certain metrics on the mini-Imagenet\n5-way 5-shot classification task. We further propose a simple and effective way\nof conditioning a learner on the task sample set, resulting in learning a\ntask-dependent metric space. Moreover, we propose and empirically test a\npractical end-to-end optimization procedure based on auxiliary task co-training\nto learn a task-dependent metric space. The resulting few-shot learning model\nbased on the task-dependent scaled metric achieves state of the art on\nmini-Imagenet. We confirm these results on another few-shot dataset that we\nintroduce in this paper based on CIFAR100. Our code is publicly available at\nhttps://github.com/ElementAI/TADAM.","[arxiv.Result.Author('Boris N. Oreshkin'), arxiv.Result.Author('Pau Rodriguez'), arxiv.Result.Author('Alexandre Lacoste')]",http://arxiv.org/abs/1805.10123v4,40.610191
2,Exploring the Limits of Large Scale Pre-training,"Recent developments in large-scale machine learning suggest that by scaling\nup data, model size and training time properly, one might observe that\nimprovements in pre-training would transfer favorably to most downstream tasks.\nIn this work, we systematically study this phenomena and establish that, as we\nincrease the upstream accuracy, the performance of downstream tasks saturates.\nIn particular, we investigate more than 4800 experiments on Vision\nTransformers, MLP-Mixers and ResNets with number of parameters ranging from ten\nmillion to ten billion, trained on the largest scale of available image data\n(JFT, ImageNet21K) and evaluated on more than 20 downstream image recognition\ntasks. We propose a model for downstream performance that reflects the\nsaturation phenomena and captures the nonlinear relationship in performance of\nupstream and downstream tasks. Delving deeper to understand the reasons that\ngive rise to these phenomena, we show that the saturation behavior we observe\nis closely related to the way that representations evolve through the layers of\nthe models. We showcase an even more extreme scenario where performance on\nupstream and downstream are at odds with each other. That is, to have a better\ndownstream performance, we need to hurt upstream accuracy.","[arxiv.Result.Author('Samira Abnar'), arxiv.Result.Author('Mostafa Dehghani'), arxiv.Result.Author('Behnam Neyshabur'), arxiv.Result.Author('Hanie Sedghi')]",http://arxiv.org/abs/2110.02095v1,40.749702


In [11]:
# Exmaple 2: Search via a concept you're reading
query = """
What is the general idea behind self-supervised learning.
"""

result = search_table(query)

result.pop("vector")
display(HTML(result.to_html()))

C:\Users\FISCLOUDS\AppData\Local\Temp\ipykernel_26876\404315364.py:9: DeprecatedWarning: to_df is deprecated as of 0.3.1 and will be removed in 0.4.0. Use the bar function instead
  return tbl.search(embs.tolist()[0]).limit(3).to_df()


,title,summary,authors,url,_distance
0,Unsupervised Learning via Meta-Learning,"A central goal of unsupervised learning is to acquire representations from\nunlabeled data or experience that can be used for more effective learning of\ndownstream tasks from modest amounts of labeled data. Many prior unsupervised\nlearning works aim to do so by developing proxy objectives based on\nreconstruction, disentanglement, prediction, and other metrics. Instead, we\ndevelop an unsupervised meta-learning method that explicitly optimizes for the\nability to learn a variety of tasks from small amounts of data. To do so, we\nconstruct tasks from unlabeled data in an automatic way and run meta-learning\nover the constructed tasks. Surprisingly, we find that, when integrated with\nmeta-learning, relatively simple task construction mechanisms, such as\nclustering embeddings, lead to good performance on a variety of downstream,\nhuman-specified tasks. Our experiments across four image datasets indicate that\nour unsupervised meta-learning approach acquires a learning algorithm without\nany labeled data that is applicable to a wide range of downstream\nclassification tasks, improving upon the embedding learned by four prior\nunsupervised learning methods.","[arxiv.Result.Author('Kyle Hsu'), arxiv.Result.Author('Sergey Levine'), arxiv.Result.Author('Chelsea Finn')]",http://arxiv.org/abs/1810.02334v6,34.913574
1,Local contrastive loss with pseudo-label based self-training for semi-supervised medical image segmentation,"Supervised deep learning-based methods yield accurate results for medical\nimage segmentation. However, they require large labeled datasets for this, and\nobtaining them is a laborious task that requires clinical expertise.\nSemi/self-supervised learning-based approaches address this limitation by\nexploiting unlabeled data along with limited annotated data. Recent\nself-supervised learning methods use contrastive loss to learn good global\nlevel representations from unlabeled images and achieve high performance in\nclassification tasks on popular natural image datasets like ImageNet. In\npixel-level prediction tasks such as segmentation, it is crucial to also learn\ngood local level representations along with global representations to achieve\nbetter accuracy. However, the impact of the existing local contrastive\nloss-based methods remains limited for learning good local representations\nbecause similar and dissimilar local regions are defined based on random\naugmentations and spatial proximity; not based on the semantic label of local\nregions due to lack of large-scale expert annotations in the\nsemi/self-supervised setting. In this paper, we propose a local contrastive\nloss to learn good pixel level features useful for segmentation by exploiting\nsemantic label information obtained from pseudo-labels of unlabeled images\nalongside limited annotated images. In particular, we define the proposed loss\nto encourage similar representations for the pixels that have the same\npseudo-label/ label while being dissimilar to the representation of pixels with\ndifferent pseudo-label/label in the dataset. We perform pseudo-label based\nself-training and train the network by jointly optimizing the proposed\ncontrastive loss on both labeled and unlabeled sets and segmentation loss on\nonly the limited labeled set. We evaluated on three public cardiac and prostate\ndatasets, and obtain high segmentation performance.","[arxiv.Result.Author('Krishna Chaitanya'), arxiv.Result.Author('Ertunc Erdil'), arxiv.Result.Author('Neerav Karani'), arxiv.Result.Author('Ender Konukoglu')]",http://arxiv.org/abs/2112.09645v1,35.332321
2,Universum GANs: Improving GANs through contradictions,"Limited availability of labeled-data makes any supervised learning problem\nchallenging. Alternative learning settings like semi-supervised and universum\nlearning alleviate the dependency on labeled data, but still require a large\namount of unlabeled data, which may be un

# Full Text Search
In text retrieval, full-text search refers to techniques for searching a single computer-stored document or a collection in a full-text database. Full-text search is distinguished from searches based on metadata or on parts of the original texts represented in databases

LanceDB now provides **experimental** support for full text search. This is currently Python only. We plan to push the integration down to Rust in the future to make this available for JS as well.


In [12]:
!pip install tantivy@git+https://github.com/quickwit-oss/tantivy-py#164adc87e1a033117001cf70e38c82a53014d985

  Cloning https://github.com/quickwit-oss/tantivy-py to c:\users\fisclouds\appdata\local\temp\pip-install-40gpfser\tantivy_a0964f9b15de4b8a97fef4cabf7501ac
  Resolved https://github.com/quickwit-oss/tantivy-py to commit a47fcfb3a6ad3fa2fca76513bd52d840ff15c596
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for tantivy: filename=tantivy-0.20.1-cp311-none-win_amd64.whl size=2220637 sha256=9f6c3de0440792ff51dd28c43fda6c9c677eca40aa52d9abfe8adbfc85f7ba5f
  Stored in directory: C:\Users\FISCLOUDS\AppData\Local\Temp\pip-ephem-wheel-cache-jq0gtsdx\wheels\a0\4b\51\01cf34433cfc7c7d14540fad11482f0d06b197c895eb874a83
Successfully built tantivy


  Running command git clone --filter=blob:none --quiet https://github.com/quickwit-oss/tantivy-py 'C:\Users\FISCLOUDS\AppData\Local\Temp\pip-install-40gpfser\tantivy_a0964f9b15de4b8a97fef4cabf7501ac'


### Build FTS index for the summary
Here, we're building the FTS index using python bindings for tantivy. You can also build the index for any other text column. A full-text index stores information about significant words and their location within one or more columns of a database table

In [13]:
# This cell might take a few mins
tbl.create_fts_index("summary")

In [14]:
## FTS via title
result = (
    tbl.search("What is the general idea behind self-supervised learning.")
    .limit(10)
    .to_pandas()
)

result.pop("vector")

display(HTML(result.to_html()))

C:\Users\FISCLOUDS\AppData\Local\Temp\ipykernel_26876\943706364.py:2: DeprecatedWarning: to_df is deprecated as of 0.3.1 and will be removed in 0.4.0. Use the bar function instead
  result = tbl.search("What is the general idea behind self-supervised learning.").limit(10).to_df()


,title,summary,authors,url,score
0,Local contrastive loss with pseudo-label based self-training for semi-supervised medical image segmentation,"Supervised deep learning-based methods yield accurate results for medical\nimage segmentation. However, they require large labeled datasets for this, and\nobtaining them is a laborious task that requires clinical expertise.\nSemi/self-supervised learning-based approaches address this limitation by\nexploiting unlabeled data along with limited annotated data. Recent\nself-supervised learning methods use contrastive loss to learn good global\nlevel representations from unlabeled images and achieve high performance in\nclassification tasks on popular natural image datasets like ImageNet. In\npixel-level prediction tasks such as segmentation, it is crucial to also learn\ngood local level representations along with global representations to achieve\nbetter accuracy. However, the impact of the existing local contrastive\nloss-based methods remains limited for learning good local representations\nbecause similar and dissimilar local regions are defined based on random\naugmentations and spatial proximity; not based on the semantic label of local\nregions due to lack of large-scale expert annotations in the\nsemi/self-supervised setting. In this paper, we propose a local contrastive\nloss to learn good pixel level features useful for segmentation by exploiting\nsemantic label information obtained from pseudo-labels of unlabeled images\nalongside limited annotated images. In particular, we define the proposed loss\nto encourage similar representations for the pixels that have the same\npseudo-label/ label while being dissimilar to the representation of pixels with\ndifferent pseudo-label/label in the dataset. We perform pseudo-label based\nself-training and train the network by jointly optimizing the proposed\ncontrastive loss on both labeled and unlabeled sets and segmentation loss on\nonly the limited labeled set. We evaluated on three public cardiac and prostate\ndatasets, and obtain high segmentation performance.","[arxiv.Result.Author('Krishna Chaitanya'), arxiv.Result.Author('Ertunc Erdil'), arxiv.Result.Author('Neerav Karani'), arxiv.Result.Author('Ender Konukoglu')]",http://arxiv.org/abs/2112.09645v1,7.476211
1,Multi-Scale Representation Learning for Spatial Feature Distributions using Grid Cells,"Unsupervised text encoding models have recently fueled substantial progress\nin NLP. The key idea is to use neural networks to convert words in texts to\nvector space representations based on word positions in a sentence and their\ncontexts, which are suitable for end-to-end training of downstream tasks. We\nsee a strikingly similar situation in spatial analysis, which focuses on\nincorporating both absolute positions and spatial contexts of geographic\nobjects such as POIs into models. A general-purpose representation model for\nspace is valuable for a multitude of tasks. However, no such general model\nexists to date beyond simply applying discretization or feed-forward nets to\ncoordinates, and little effort has been put into jointly modeling distributions\nwith vastly different characteristics, which commonly emerges from GIS data.\nMeanwhile, Nobel Prize-winning Neuroscience research shows that grid cells in\nmammals provide a multi-scale periodic representation that functions as a\nmetric for location encoding and is critical for recognizing places and for\npath-integration. Therefore, we propose a representation learning model called\nSpace2Vec to encode the absolute positions and spatial relationships of places.\nWe conduct experiments on two real-world geographic data for two different\ntasks: 1) predicting types of POIs given their positions and context, 2) image\nclassification leveraging their geo-locations. Results show that because of its\nmulti-scale representations, Space2Vec outperforms well-established ML\napproaches such as RBF kernels, multi-layer feed-forward nets, and til

### Analysing OpenCLIP embeddings on Nomic
Atlas is a platform for interacting with both small and internet scale unstructured datasets.

Atlas enables you to:
* Store, update and organize multi-million point datasets of unstructured text, images and embeddings.
* Visually interact with embeddings of your data from a web browser.
* Operate over unstructured data and embeddings with topic modeling, semantic duplicate clustering and semantic search.
* Generate high dimensional and two-dimensional embeddings of your data.

In [15]:
!pip install nomic --q

In [16]:
!nomic login

                        Authenticate with the Nomic API                        
                       https://atlas.nomic.ai/cli-login                        
 Click the above link to retrieve your access token and then run `nomic login  
                                   [token]`                                    


In [21]:
!nomic login #Paste your token from Nomic Ai cli login -- here

In [20]:
from nomic import atlas
import numpy as np

# Get pandas dataframe from lancedb table
df = tbl.to_pandas()

# get embeddings from df
embs = np.array(df.pop("vector").to_list())

project = atlas.map_embeddings(embeddings=embs, data=df.to_dict("records"))
print()

2023-10-15 12:45:42.522 | WARNING  | nomic.atlas:map_embeddings:95 - An ID field was not specified in your data so one was generated for you in insertion order.
2023-10-15 12:45:49.237 | INFO     | nomic.project:_create_project:790 - Creating project `voracious-remark` in organization `kaushalc64`
2023-10-15 12:45:51.920 | INFO     | nomic.atlas:map_embeddings:111 - Uploading embeddings to Atlas.
1it [00:01,  1.88s/it]
2023-10-15 12:45:53.844 | INFO     | nomic.project:_add_data:1422 - Upload succeeded.
2023-10-15 12:45:53.849 | INFO     | nomic.atlas:map_embeddings:130 - Embedding upload succeeded.
2023-10-15 12:45:57.000 | INFO     | nomic.project:create_index:1132 - Created map `voracious-remark` in project `voracious-remark`: https://atlas.nomic.ai/map/9e13dcd5-15e1-4449-9005-93292f739c2c/aa195bbd-11f6-4813-8435-6468192274cc
2023-10-15 12:45:57.000 | INFO     | nomic.atlas:map_embeddings:143 - voracious-remark: https://atlas.nomic.ai/map/9e13dcd5-15e1-4449-9005-93292f739c2c/aa195bb

The visualizations are very interesting and is worth exploring more. IN preliminary analysis, you can see that it succesfully creates clusters of similar types of papers. There are a few things that can be done next like comparing embeddings on various openclip models sizes and datasets. 
<img width="1433" alt="Screenshot 2023-08-24 at 3 47 51 PM" src="https://github.com/lancedb/vectordb-recipes/assets/15766192/34ef88a3-2925-4450-abcd-1abc350ef3e4">